In [22]:
import os
import PyPDF2
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/mertozlutras/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/mertozlutras/nltk_data...


True

In [23]:
wnl = WordNetLemmatizer()

In [9]:

# Define the directory containing the PDF files
directory = 'annual_reports_III'

# Define the list of keywords to search for
candidates_scope1 = 'Direct emissions, on-site emissions, on site emissions, combustion emissions, fuel consumption, stationary sources, fugitive emissions, process emissions, greenhouse gas emissions, carbon footprint, Clean Air Act, emissions reductions, carbon tax, carbon offsetting, carbon credits, carbon trading, carbon footprint reduction, carbon neutral, carbon management, carbon accounting'
candidates_scope1 = candidates_scope1.split(', ')

candidates_scope2 = 'Indirect emissions, purchased electricity, grid-purchased electricity, energy consumption, greenhouse gas emissions, carbon footprint, Renewable Energy Certificates (RECs), power purchase agreements (PPAs), energy efficiency, demand-side management, energy conservation, carbon offsetting, carbon credits, carbon trading, carbon footprint reduction, carbon neutral, carbon management, carbon accounting'
candidates_scope2 = candidates_scope2.split(', ')

candidates_scope3 = 'Indirect emissions, supply chain emissions, upstream emissions, downstream emissions, transportation emissions, distribution emissions, use phase emissions, end-of-life emissions, embodied carbon, life cycle assessment (LCA), carbon footprint, carbon offsetting, carbon credits, carbon trading, carbon footprint reduction, carbon neutral, carbon management, carbon accounting'
candidates_scope3 = candidates_scope3.split(', ')

In [50]:
# Cleans the sentences and gives you clean things to search for in pdfs

def clean_keyword_sentences(candidates):
    keywords_scope = []
    # We get every phrase from the keywords list
    for phrase in candidates:
        # each keyword from the keywords list will be lemmatized
        keywords = []
        for keyword in phrase.split():
            keywords.append(wnl.lemmatize(keyword.lower()))
        keywords_scope.append(' '.join(keywords))
        
    return keywords_scope

In [54]:
keywords = clean_keyword_sentences(candidates_scope1)

In [52]:
keywords_scope1

['direct emission',
 'on-site emission',
 'on site emission',
 'combustion emission',
 'fuel consumption',
 'stationary source',
 'fugitive emission',
 'process emission',
 'greenhouse gas emission',
 'carbon footprint',
 'clean air act',
 'emission reduction',
 'carbon tax',
 'carbon offsetting',
 'carbon credit',
 'carbon trading',
 'carbon footprint reduction',
 'carbon neutral',
 'carbon management',
 'carbon accounting']

In [72]:
# Iterate over each file in the directory

# We input which scope we are looking for, it uses the correct search keyword list for this specific scope 
def retrieve_examples_from_all_files(scope:int):
    if scope == 1:
        search_for_list = candidates_scope1
    elif scope == 2:
        search_for_list = candidates_scope2
    elif scope == 3:
        search_for_list = candidates_scope3
    else:
        print('Scope should be 1,2 or 3')

    keywords = clean_keyword_sentences(search_for_list)

    examples_to_annotate = []
    scope = scope

    counter = 1
    for filename in os.listdir(directory):
        if filename.endswith('.pdf'):
            # Open the PDF file
            with open(os.path.join(directory, filename), 'rb') as pdf_file:
                pdf_reader = PyPDF2.PdfReader(pdf_file)

                # Iterate over each page
                for page_num in range(len(pdf_reader.pages)):
                    page = pdf_reader.pages[page_num]
                    text = page.extract_text()

                    # Clean the text by removing newline characters
                    text = ' '.join(text.split())

                    # lemmatize the words
                    text = ' '.join([wnl.lemmatize(keyword.lower()) for keyword in text.split()])

                    # Iterate over each keyword
                    for keyword in keywords:
                        # Search for keyword in the text
                        if keyword in text:
                            print(f'{keyword} found on page {page_num+1} of {filename}')

                            # we define the dictionary item to add to our list
                            item = {'filename': filename,
                                   'page': page_num,
                                   'text': text,
                                   'scope': scope}

                            examples_to_annotate.append(item)
        
    return examples_to_annotate

In [73]:
examples_scope1 = retrieve_examples_from_all_files(scope=1)

fuel consumption found on page 33 of 0XXV-GB_2019_annual-report.pdf
fuel consumption found on page 37 of 0XXV-GB_2019_annual-report.pdf
carbon footprint found on page 46 of 0XXV-GB_2019_annual-report.pdf
greenhouse gas emission found on page 141 of 0XXV-GB_2019_annual-report.pdf
greenhouse gas emission found on page 143 of 0XXV-GB_2019_annual-report.pdf
greenhouse gas emission found on page 11 of CAP-FR_2019_universal-registration-document.pdf
greenhouse gas emission found on page 155 of CAP-FR_2019_universal-registration-document.pdf
carbon footprint found on page 155 of CAP-FR_2019_universal-registration-document.pdf
carbon accounting found on page 155 of CAP-FR_2019_universal-registration-document.pdf
greenhouse gas emission found on page 158 of CAP-FR_2019_universal-registration-document.pdf
carbon footprint found on page 159 of CAP-FR_2019_universal-registration-document.pdf
emission reduction found on page 159 of CAP-FR_2019_universal-registration-document.pdf
carbon footprint fo

KeyboardInterrupt: 